In [65]:
import requests, pdb, json, uuid, nbformat, psycopg2, datetime, os
import pandas as pd

PLAID_CLIENT_ID=os.environ['PLAID_CLIENT_ID']
PLAID_SECRET=os.environ['PLAID_SECRET']

# Options: 'sandbox', 'development', 'production'
PLAID_ENV = 'os.environ['PLAID_ENV']
PLAID_BASE_URL = f'https://{PLAID_ENV}.plaid.com'


In [8]:
def plaid_post(endpoint, payload) -> json:
    """ 
    Helper function to make POST requests to Plaid API
    """
    url = f"{PLAID_BASE_URL}/{endpoint}"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    return response.json()


def get_accounts_df(accounts_response) -> pd.DataFrame:
   """
   Returns the accounting information in a pandas data frame
   """
   return pd.json_normalize(accounts_response, sep='_')

def create_uuid() -> str:
    """
    Creates uuid if not exist
    """

    return str(uuid.uuid4())

def about():
    print("Jupyter Personal Finance" , \
          "\nClient       : ", PLAID_CLIENT_ID, \
          "\nEnvironment  : ", PLAID_ENV)


## First time setup
1. Follow on screeen prompts
2. Need to set up your email with Plaid API
3. Receive link to login with Plaid API
4. Authenticate with bank providers
5. Receive 'access tokens'. This is stored locally and will be used to query your data 

In [6]:
email = input('Your Email - (name@email.com)')
phone = input('Your Phone - (+1 123 4567890)')

In [ ]:
# First Time Token Authentication

# Step 1: Obtain a Link Token
payload = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "client_name": "Juypter Notebook",
    "country_codes": ["US", "CA"],
    "language": "en",
    "user": {"client_user_id": 'Still Testing', "phone_number":phone, "email_address": email},
    "hosted_link": {
  },
    "products": ["transactions"]
}
link_token_response = plaid_post("link/token/create", payload)
link_token=link_token_response['link_token']

# First time login


In [ ]:
print('Navigate to this link and authenticate: \n: ', link_token_response['hosted_link_url'])

In [14]:
payload = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "link_token": link_token
}
link_token_details = plaid_post("link/token/get", payload)
#print("Link Token Details:", json.dumps(link_token_details, indent=2))

In [ ]:
print(link_token_details)

In [ ]:
# Step 2: Exchange Public Token for Access Token (Assumes you've completed Plaid Link flow)
public_token = link_token_details['link_sessions'][0]['results']['item_add_results'][0]['public_token']  # Replace with public_token obtained from Plaid Link flow
payload = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "public_token": public_token
}
exchange_response = plaid_post("item/public_token/exchange", payload)
access_token = exchange_response.get("access_token")
print("Access Token:", access_token)

print("Access token has been generated for ", email, " with the bank", )


In [89]:
# Step 3: Fetch Account Information
payload = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "access_token": access_token
}
accounts_response = plaid_post("accounts/get", payload)

# Step 4: Fetch Transactions
transactions = []
payload = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "access_token": access_token,
    "start_date": "2024-01-01",
    "end_date": "2024-12-31",
    "options": {
        "count": 100,  
        "offset": 0   
    }
}

while True:
    transactions_response = plaid_post("transactions/get", payload)
    transactions.extend(transactions_response.get("transactions", []))
    
    # Check if there are more transactions to fetch
    if len(transactions) >= transactions_response.get("total_transactions", 0):
        break

    # Update the offset for the next batch
    payload["options"]["offset"] += 100




In [63]:
accounts_df = get_accounts_df(accounts_response['accounts'])
transactions_df = pd.json_normalize(transactions)


In [ ]:
# accounts
conn = psycopg2.connect(
    host="localhost",
    database="finances",
    user="admin",
    password="admin"
)

cur = conn.cursor()
for index, account in accounts_df.iterrows():
    cur.execute("""
        INSERT INTO accounts (
            account_id, 
            mask,
            name,
            official_name,
            persistent_account_id,
            subtype,
            type,
            user_email,
            user_phone,
            plaid_access_token
        ) 
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (account_id) 
        DO UPDATE SET
            mask = EXCLUDED.mask,
            name = EXCLUDED.name,
            official_name = EXCLUDED.official_name,
            persistent_account_id = EXCLUDED.persistent_account_id,
            subtype = EXCLUDED.subtype,
            type = EXCLUDED.type,
            user_email = EXCLUDED.user_email,
            user_phone = EXCLUDED.user_phone,
            plaid_access_token = EXCLUDED.plaid_access_token;
    """, (
        account['account_id'],
        account['mask'],
        account['name'],
        account['official_name'],
        account['persistent_account_id'],
        account['subtype'],
        account['type'],
        email,
        phone,
        access_token,
    ))

# Commit the transaction and close the cursor and connection
conn.commit()
cur.close()
conn.close()


In [88]:
## ACCOUNT BALANCE HISTORY
conn = psycopg2.connect(
    host="localhost",
    database="finances",
    user="admin",
    password="admin"
)

cur = conn.cursor()

for index, account in accounts_df.iterrows():
    cur.execute("""
        INSERT INTO accounts_balance_history (
            account_id, 
            balances_available,
            balances_current,
            balances_iso_currency_code,
            balances_limit,
            balances_unofficial_currency_code,
            balances_datetime
        ) 
        VALUES (
            %s, %s, %s, %s, %s, %s, %s
        );
    """, (
        account['account_id'],
        account['balances_available'],
        account['balances_current'],
        account['balances_iso_currency_code'],
        account['balances_limit'],
        account['balances_unofficial_currency_code'],
        datetime.datetime.now(),
    ))

# Commit the transaction and close the cursor and connection
conn.commit()
cur.close()
conn.close()
